# Create h5 dataset from audio files and json text_metadata

In [1]:
!PYTHONPATH=. python data/create_h5_dataset.py \
-audio-dir data/example_audio_files \
--target-dir data/example_h5_files \
--json-path data/example_text_feat_jsons/example_text_feat.json 

    torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
    To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
    TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
----------------------------------
Parsing audio folder data/example_audio_files
num audio paths: 7
num valid audio paths: 7
Dataset created


0 : Parsing DefineDancing
------saving text feat meta data------
Got metadata of audio DefineDancing
Found madmom_key feature
Found madmom_tempo feature
Found text description
Got text This is a beautiful instrumental piece of music. It has a calming and soothing effect on the listener's min

# Load h5 dataset

In [1]:
from data.h5_dataset import DacEncodecClapDatasetH5

def display_chunk_data(chunk_datadict):
    for key in chunk_datadict.keys():
        this_component = chunk_datadict[key]
        if hasattr(this_component, "shape"):
            if len(this_component.shape) == 0:
                to_show = this_component
            else:
                to_show = f"shape = {this_component.shape}, max = {this_component.max()}, min = {this_component.min()}"
        else:
            to_show = this_component
    
        print(f"- {key}: {to_show}")

In [2]:
h5_dir = "data/example_h5_files"
dac_frame_len = 2320
dataset = DacEncodecClapDatasetH5(
    h5_dir=h5_dir,
    dac_frame_len=dac_frame_len,
)

Reading dac h5 file metadata...
Num files: 7


    `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884


In [3]:
chunk_data = dataset[0] # it should be different in each run because the dataset is randomized
display_chunk_data(chunk_data)

- madmom_key: 17
- madmom_tempo: 100
- text: This is a jazz piece with big band. It features trumpets as the main instrument and is characterized by a swing tempo and a jazzy melody. The piece is in 4/4 time and has a groovy rhythm that is perfect for sipping cocktails and relaxing with friends.
- text_clap: shape = (512,), max = 0.13925595581531525, min = -0.13986632227897644
- tags_text: genres: jazz, swing; instrument: trumpets, saxphones, trombones, piano, drums, bass
- tags_text_clap: shape = (512,), max = 0.13922543823719025, min = -0.13837091624736786
- chatgpt_text_clap: shape = (512,), max = 0.13962218165397644, min = -0.13861507177352905
- name: MoscowNights_chunk_3
- dac_rvq: shape = (9, 2320), max = 1023, min = 0
- dac_latents: shape = (72, 2320), max = 23.53624153137207, min = -21.052751541137695
- encodec_rvq: shape = (4, 1344), max = 2047, min = 0
- encodec_latents: shape = (512, 1344), max = 41.90758514404297, min = -39.326751708984375
- audio_clap: shape = (512,), max 

In [4]:
h5_dir = "data/example_h5_files"
dac_frame_len = 2320
dataset2 = DacEncodecClapDatasetH5(
    h5_dir=h5_dir,
    dac_frame_len=dac_frame_len,
    random_load=False
)

Reading dac h5 file metadata...
Num files: 7
Num chunks: 38


In [5]:
chunk_data = dataset2[0] # it should always be the same in each run
display_chunk_data(chunk_data)

- madmom_key: 17
- madmom_tempo: 100
- text: This is a jazz piece with big band. It features trumpets as the main instrument and is characterized by a swing tempo and a jazzy melody. The piece is in 4/4 time and has a groovy rhythm that is perfect for sipping cocktails and relaxing with friends.
- text_clap: shape = (512,), max = 0.13925595581531525, min = -0.13986632227897644
- tags_text: genres: jazz, swing; instrument: trumpets, saxphones, trombones, piano, drums, bass
- tags_text_clap: shape = (512,), max = 0.13922543823719025, min = -0.13837091624736786
- chatgpt_text_clap: shape = (512,), max = 0.13962218165397644, min = -0.13861507177352905
- name: MoscowNights_chunk_0
- dac_rvq: shape = (9, 2320), max = 1023, min = 0
- dac_latents: shape = (72, 2320), max = 19.940122604370117, min = -19.030113220214844
- encodec_rvq: shape = (4, 1344), max = 2047, min = 1
- encodec_latents: shape = (512, 1344), max = 41.90758514404297, min = -38.61660385131836
- audio_clap: shape = (512,), max 

# Normalize the dac latents (not used currently)

In [6]:
import torch

DAC_CODEBOOK_MEANS = [-0.0357, -0.0271, -0.0347, -0.0220, -0.0334, -0.0355, -0.0365, -0.0300, -0.0244]# [0.58, -0.11, -0.18, -0.04, -0.23, -0.06, 0.08, 0, 0.03]
DAC_CODEBOOK_STDS = [3.2464, 3.2818, 3.2376, 3.2643, 3.2814, 3.2808, 3.2847, 3.2798, 3.2684] # [4.95, 4.00, 3.61, 3.41, 3.24, 3.16, 3.06, 2.93, 2.79]
DAC_DIM_SINGLE = 8
DAC_N_CODEBOOKS = len(DAC_CODEBOOK_MEANS)
DAC_DIMS = DAC_DIM_SINGLE * DAC_N_CODEBOOKS
DAC_CODEBOOK_MEANS_BY_DIM = []
DAC_CODEBOOK_STDS_BY_DIM = []
for i_codebook in range(DAC_N_CODEBOOKS):
    DAC_CODEBOOK_MEANS_BY_DIM += [DAC_CODEBOOK_MEANS[i_codebook]]*DAC_DIM_SINGLE
    DAC_CODEBOOK_STDS_BY_DIM += [DAC_CODEBOOK_STDS[i_codebook]]*DAC_DIM_SINGLE
DAC_CODEBOOK_MEANS_BY_DIM = torch.tensor(DAC_CODEBOOK_MEANS_BY_DIM)
DAC_CODEBOOK_MEANS_PRIMARY_BY_DIM = DAC_CODEBOOK_MEANS_BY_DIM[:DAC_DIM_SINGLE]
DAC_CODEBOOK_MEANS_SECONDARY_BY_DIM = DAC_CODEBOOK_MEANS_BY_DIM[DAC_DIM_SINGLE:]
DAC_CODEBOOK_STDS_BY_DIM = torch.tensor(DAC_CODEBOOK_STDS_BY_DIM)
DAC_CODEBOOK_STDS_PRIMARY_BY_DIM = DAC_CODEBOOK_MEANS_BY_DIM[:DAC_DIM_SINGLE]
DAC_CODEBOOK_STDS_SECONDARY_BY_DIM = DAC_CODEBOOK_MEANS_BY_DIM[DAC_DIM_SINGLE:]

def shape_dac_mean_std(latents: torch.Tensor):
    shape = latents.shape
    if len(shape) == 3:
        assert shape[1] == DAC_DIMS
        mean = DAC_CODEBOOK_MEANS_BY_DIM.unsqueeze(0).unsqueeze(-1)
        std = DAC_CODEBOOK_STDS_BY_DIM.unsqueeze(0).unsqueeze(-1)
    elif len(shape) == 2:
        assert shape[0] == DAC_DIMS
        mean = DAC_CODEBOOK_MEANS_BY_DIM.unsqueeze(-1)
        std = DAC_CODEBOOK_STDS_BY_DIM.unsqueeze(-1)
    elif len(shape) == 1:
        assert shape[0] == DAC_DIMS
        mean = DAC_CODEBOOK_MEANS_BY_DIM[:]
        std = DAC_CODEBOOK_STDS_BY_DIM
    dtype = latents.dtype
    device = latents.device
    return mean.to(dtype = dtype, device = device), std.to(dtype = dtype, device = device)

def dac_latents_normalize(latents: torch.Tensor):
    mean, std = shape_dac_mean_std(latents)
    return (latents - mean) / std

def dac_latents_denormalize(latents: torch.Tensor):
    mean, std = shape_dac_mean_std(latents)
    return latents * std + mean

In [7]:
dac_latents = []
for _ in range(3000):
    dac_latents += [torch.tensor(dataset[0]["dac_latents"])]
dac_latents = torch.stack(dac_latents, dim=0) # (B, 72, L)
print(dac_latents.shape)
dac_latents = dac_latents.view(72, -1) # (72, BL)
print(dac_latents.shape)
dac_latents_chunked = torch.stack(torch.chunk(dac_latents, 8, dim=0), dim=-1) # (9, BL, 8)
print(dac_latents_chunked.shape)
dac_latents_chunked = dac_latents_chunked.view(9, -1) # (9, 8BL)
print(dac_latents_chunked.shape)

dac_latents_mean = dac_latents_chunked.mean(-1) # (9,)
dac_latents_std = dac_latents_chunked.std(-1) # (9,)

print(f"mean = {dac_latents_mean}")
print(f"std = {dac_latents_std}")

normalized_dac_latents = dac_latents_normalize(dac_latents)
normalized_dac_latents_chunked = torch.stack(torch.chunk(normalized_dac_latents, 8, dim=0), dim=-1) # (9, BL, 8)
normalized_dac_latents_chunked = normalized_dac_latents_chunked.view(9, -1) # (9, 8BL)

normalized_dac_latents_mean = normalized_dac_latents_chunked.mean(-1) # (9,)
normalized_dac_latents_std = normalized_dac_latents_chunked.std(-1) # (9,)

print(f"normalized mean = {normalized_dac_latents_mean}")
print(f"normalized std = {normalized_dac_latents_std}")

torch.Size([3000, 72, 2320])
torch.Size([72, 6960000])
torch.Size([9, 6960000, 8])
torch.Size([9, 55680000])
mean = tensor([-0.0328, -0.0322, -0.0322, -0.0291, -0.0276, -0.0292, -0.0312, -0.0320,
        -0.0303])
std = tensor([3.2853, 3.2650, 3.2705, 3.2661, 3.2786, 3.2693, 3.2761, 3.2620, 3.2602])
normalized mean = tensor([-2.8018e-04, -3.0361e-04, -5.6867e-04,  4.3056e-04,  9.5939e-04,
         9.0058e-04, -2.0423e-04, -1.5230e-04,  4.4813e-05])
normalized std = tensor([1.0048, 0.9991, 1.0009, 0.9994, 1.0033, 0.9998, 1.0008, 0.9982, 0.9963])
